<a href="https://colab.research.google.com/github/Klabauterkerl/pokedream/blob/main/pokedream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PokéDream

<img src="https://www.cartonionline.com/wordpress/wp-content/uploads/2022/10/Pikachu.jpg" align="right" width="300">

After binging Pokémon content, where you spent all your free time playing the games, watching the anime, and collecting the cards, you realize that something has changed about you.

Now every time you close your eyes you think about these little critters. Every time a bush is rustling in your peripheral vision you prepare yourself for battle. You even started to see Pokémon in your everyday life, imagining that the birds outside their window were Pidgeys and the squirrels in the park were Pikachu.

But what is most affected are your dreams. No matter the story, it is always about Pokémon. But as dreams are they blend everything together, sometimes the Pokémon speak, sometimes they do cruel things no Pokémon in their right mind would do.

Now another restless dreamfilled night begins as you close your eyes...


In [ ]:
#@title Update your PokéDex
%%capture
!pip install -q diffusers transformers accelerate scipy safetensors torch
!pip3 install -q openai
!pip install -q nltk


# Characters:

These are the main characters of your dreams (left to right):

*   Charmander: an adorable reptilian Pokémon, with a bright flame at the tip of its tail, giving off a warm and friendly glow. (Fire-Type)
*   Bulbasaur: a lovely creature with a charming bulb on its back that blossoms into a beautiful flower as it evolves. (Grass-Type)
*   Pikachu: an adorable and lovable yellow rodent with rosy cheeks and a lightning bolt-shaped tail. (Electric-Type)
*   Squirtle: a cute and playful water-type Pokémon with a signature shell and adorable blue eyes. (Water-Type)



<img src="https://assets.pokemon.com/assets/cms2/img/pokedex/full/004.png" width="200"> <img src="https://archives.bulbagarden.net/media/upload/f/fb/0001Bulbasaur.png" width="200"> <img src="https://assets.stickpng.com/images/580b57fcd9996e24bc43c325.png" width="200"> <img src="https://assets.stickpng.com/images/580b57fcd9996e24bc43c32a.png" width="200">



In [ ]:
#@title Dream up a story

from importlib import invalidate_caches
#from tkinter.constants import N
import openai
#from collections import defaultdict
import nltk
from nltk.tokenize import word_tokenize
#from nltk.probability import FreqDist
#nltk.download('punkt', quiet=True)
#nltk.download('averaged_perceptron_tagger', quiet=True)

class TextGene:
  def __init__(self, author, key, n1, n2, n3, sentence, concept, ending, model, init_s):
    self.author = author
    self.key = key
    self.n1 = n1
    self.n2 = n2
    self.n3 = n3
    self.sentence = sentence
    self.concept = concept
    self.ending = ending
    self.model = model
    self.init_s = init_s
    self.story = ''
    self.write_story()
    self.title = ''
    self.get_title()
    self.titles = []
    self.texts = []
    self.chapters = {}
    self.final_story = ''
    self.story_in_chapters = []
    self.get_chapter()
    self.prompts = []
    self.generate_prompts()
    self.selected_concept = ''
    self.select_music()
    self.name_list = []
    self.pokemon_list()


  def write_story(self):
    openai.api_key = self.key 
    if self.sentence == '':
        prompt_text = f"Write a four chapter {self.concept} story {self.ending} with the main characters being: Pokemon {self.n1}, {self.n2}, {self.n3} {self.init_s}. Please give a chapter name for each chapter."
    else:
        prompt_text = f"Write a four chapter {self.concept} story {self.ending} with the main characters being: Pokemon {self.n1}, {self.n2}, {self.n3} {self.sentence}. Please give a chapter name for each chapter."
    completions = openai.Completion.create(
    engine=self.model,
    prompt=prompt_text,
    max_tokens=800,
    n=1,
    stop=None,
    temperature=0.5,
    )

    text = completions.choices[0].text
    index = len(text) - 1
    while text[index] != '.':
      index -= 1
    self.story = text[2:index+1]
    return self.story

  def get_title(self):
    openai.api_key = self.key 
    prompt_text = f"Write a title for the story: {self.story}"
    completions = openai.Completion.create(
    engine=self.model,
    prompt=prompt_text,
    max_tokens=40,
    n=1,
    stop=None,
    temperature=0.5,
    )
    title = completions.choices[0].text
    index = len(title)
    self.title = title[2:index]
    return self.title

  def get_chapter(self):
    prompts = self.story.split('\n')
    temp = ''
    for i in range(len(prompts)):
        if "Chapter" in prompts[i]:
            self.titles.append(prompts[i])
            if temp != '':
                self.texts.append(temp)
                temp = ''
        elif 'Epilogue' in prompts[i]:
          self.titles.append(prompts[i])
          if temp != '':
            self.texts.append(temp)
            temp = ''
        else:
            temp += prompts[i] + '\n\n'
    if temp != '':
      self.texts.append(temp)
    for i in range(len(self.titles)):
      self.chapters[self.titles[i]] = self.texts[i]
    for key, value in self.chapters.items():
      self.final_story += '{key}\n\n{value}'.format(key = key, value = value)
      self.story_in_chapters.append('{key}\n\n{value}'.format(key = key, value = value))
  
  def generate_prompts(self):
    openai.api_key = self.key
    for key, value in self.chapters.items():
      prompt_text = f'Please describe the main location and how the place looks of this text in short phrase: {value}'
      completions = openai.Completion.create(
      engine=self.model,
      prompt=prompt_text,
      max_tokens=20,
      n=1,
      stop=None,
      temperature=0.1,
      )
      text = completions.choices[0].text
      stopwords = ['pokémon', 'pokemon', 'two', 'three', 'friends']
      stopwords.append(self.n1.lower())
      stopwords.append(self.n2.lower())
      stopwords.append(self.n3.lower())
      tokens = word_tokenize(text)
      filtered_tokens = [word for word in tokens if word.lower() not in stopwords]
      prompt = ''
      for w in filtered_tokens:
        prompt += w + ' '
      self.prompts.append(prompt)

  def select_music(self):
    # Set up the OpenAI API client
    openai.api_key = self.key

    # Define the concepts and the text to analyze
    concepts = ["Happy", "Sad", "Exciting", "Scary", "Mysterious", "Surprised", "Crazy", "Touching"]
    text = self.sentence

    # Use the GPT-3 model to generate a response
    model_engine = self.model
    prompt = (f"Which concept is the text '{text}' most like? " 
              + " ".join([f"({i}) {concept}" for i, concept in enumerate(concepts)]) + "\n")
    completions = openai.Completion.create(
        engine=model_engine,
        prompt=prompt,
        max_tokens=20,
        n=1,
        stop=None,
        temperature=0.7,
    )

    # Extract the most likely completion from the response
    most_likely_completion = completions.choices[0].text

    # Extract the concept from the completion
    if most_likely_completion[-1] == '.':
      self.selected_concept = most_likely_completion[:-1]
    self.selected_concept = most_likely_completion.strip().split("\n")[-1].split(' ')[-1]
    return self.selected_concept

  def pokemon_list(self):
    self.name_list.append(self.n1)
    if self.n2 != '':
      self.name_list.append(self.n2)
      
    if self.n3 != '':
      self.name_list.append(self.n3)



#@markdown Enter your name:
author = "Kiki" #@param {type:"string"}
#@markdown Enter OpenAI-API-Key for GPT-3:
key = "" #@param {type:"string"}
#@markdown Choose Pokemon for the story:
pokemon1 = "Squirtle" #@param ["Pikachu", "Squirtle", "Charmander", "Bulbasaur"]
pokemon2 = "Charmander" #@param ["", "Pikachu", "Squirtle", "Charmander", "Bulbasaur"]
pokemon3 = "" #@param ["", "Pikachu", "Squirtle", "Charmander", "Bulbasaur"]
#@markdown Enter a short sentence about what is going to happen to the Pokemon:
sentence = "cook Japanese food at home" #@param {type:"string"}
#@markdown What type of dream?
concept = "Fancy" #@param ["", "Fancy", "Horror"]
#@markdown What type of ending?
ending = "with good ending" #@param ["", "with bad ending", "with good ending"]
model_engine = "text-davinci-003"
init_prompt = 'have a travel in Germany'
# print('Here We Go!\n\n')
if author == '':
  story = TextGene('PokéDreamer', key, pokemon1, pokemon2, pokemon3, sentence, concept, ending, model_engine, init_prompt)
else:
  story = TextGene(author, key, pokemon1, pokemon2, pokemon3, sentence, concept, ending, model_engine, init_prompt)
#print(story.final_story)

Before continuing you need to have uploaded the separately provided images and music to the filebrowser of this runtime. Just drag and drop to the top level.

In [ ]:
#@title Imagine the scene of the story
%%capture
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
import torch

#@markdown Choosing the model will influence the style of the generated images heavily.
#@markdown We suggest "prompthero/openjourney-v2" as it creates very detailed images.

#@markdown There are many more styles, click on the different models and find the style you like best [here](https://huggingface.co/models?pipeline_tag=text-to-image&sort=downloads)

#@markdown Name/Path of the initial model. (eg. "prompthero/openjourney-v2")
model_id = "prompthero/openjourney-v2" #@param {type:"string"}
#@markdown Special bonus mode (Use with caution, extremely disturbing)
nightmare_mode = False #@param {type: "boolean"}

# Use the DPMSolverMultistepScheduler (DPM-Solver++) scheduler here instead
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

if nightmare_mode:
  for idx, chapter in enumerate(story.story_in_chapters):
    prompt = chapter + ", Anime, Cartoon"
    image = pipe(prompt).images[0]
    image.save("composition" +str(idx)+".png")
else:
  for idx, chapter in enumerate(story.prompts):
    prompt = chapter + ", Background, Anime, Cartoon"
    image = pipe(prompt).images[0]
    image.save("chapter" +str(idx)+".png")

from PIL import Image
from shapely.geometry import box
import random

pokemon_images = {"Pikachu": ["pika1.png", "pika2.png"],
                "Charmander": ["char1.png", "char2.png", "char3.png"],
                "Bulbasaur": ["bulba1.png", "bulba2.png"],
                "Squirtle": ["squir1.png", "squir2.png"]}

def overlay_images(chapters, images_paths):
    for idx, _ in enumerate(chapters):
      # Select the background for the chapter
      background_path = "chapter"+str(idx)+".png"
      # Open the background image
      background = Image.open(background_path)
      background_width, background_height = background.size
      occupied_regions = []
      for image_path in images_paths:
        # Open the character image
        character = Image.open(random.choice(pokemon_images[image_path]))
        if len(images_paths) > 2:
          character = character.resize((100, 100))
        elif len(images_paths) > 1:
          character = character.resize((120, 120))
        character_width, character_height = character.size
        # Randomly select position on the bottom half of the background
        while True:
            x_offset = random.randint(0, background_width - character_width)
            y_offset = random.randint(background_height//2, background_height - character_height)
            bounding_box = box(x_offset, y_offset, x_offset + character_width, y_offset + character_height)
            overlap = False
            for region in occupied_regions:
                if region.intersects(bounding_box):
                    overlap = True
                    break
            if not overlap:
                occupied_regions.append(bounding_box)
                break

          # Position the character image on the background
        background.paste(character, (x_offset, y_offset), mask=character)

      # Save the final image
      background.save("composition"+str(idx)+".png")
if nightmare_mode:
  pass
else:
  overlay_images(story.prompts, story.name_list)


In [ ]:
#@title Watch your dream unfold
#@markdown Adjust sound levels of your Computer as music will commence when the cell has finished running.
from IPython.display import display
from ipywidgets import widgets, HBox, VBox
from IPython.core.display import display, HTML
from IPython.display import Javascript


music = widgets.Audio(value=open(story.selected_concept+'.mp3', 'rb').read())
imageA = widgets.Image(value=open('composition0.png', 'rb').read())
imageB = widgets.Image(value=open('composition1.png', 'rb').read())
imageC = widgets.Image(value=open('composition2.png', 'rb').read())
imageD = widgets.Image(value=open('composition3.png', 'rb').read())
txtA = widgets.Textarea(value=story.story_in_chapters[0], layout=widgets.Layout(height='520px', width='400px'))
txtB = widgets.Textarea(value=story.story_in_chapters[1], layout=widgets.Layout(height='520px', width='400px'))
txtC = widgets.Textarea(value=story.story_in_chapters[2], layout=widgets.Layout(height='520px', width='400px'))
txtD = widgets.Textarea(value=story.story_in_chapters[3], layout=widgets.Layout(height='520px', width='400px'))

hbox1 = HBox([music])
hbox2 = HBox([imageA, txtA])
hbox3 = HBox([imageB, txtB])
hbox4 = HBox([imageC, txtC])
hbox5 = HBox([imageD, txtD])


print("Check it out!\n")
print(story.title.upper())
print('@ ' + story.author)
display(hbox1)
display(hbox2)
display(hbox3)
display(hbox4)
display(hbox5)
Javascript("window.scrollTo(0, 0);")




In [ ]:
#@title Save the story as an image
#@markdown The file will be saved to the file browser of this Runtime, you can download it from there.
#from transformers.models.x_clip.modeling_x_clip import X_CLIP_TEXT_INPUTS_DOCSTRING
import textwrap
from PIL import Image, ImageDraw, ImageFont

# open the images
img1 = Image.open("composition0.png")
img2 = Image.open("composition1.png")
img3 = Image.open("composition2.png")
img4 = Image.open("composition3.png")

# get the size
img1_size = img1.size
img2_size = img2.size
img3_size = img3.size
img4_size = img4.size

# create new image
new_img = Image.new("RGB", (img1_size[1]*2,img1_size[0]+img2_size[0]+img3_size[0]+img4_size[1]+280), (255,255,255))
draw = ImageDraw.Draw(new_img)

# paste the image
new_img.paste(img1, (5,200))
new_img.paste(img2, (5,img1_size[0]+205))
new_img.paste(img3, (5,img1_size[0]+img2_size[0]+210))
new_img.paste(img4, (5,img1_size[0]+img2_size[0]+img3_size[0]+215))

#past the text
font_t = ImageFont.truetype("arial.ttf", 19)
font = ImageFont.truetype("arial.ttf", 15)
title = story.title.upper()
width_t, height_t = draw.textsize(title, font_t)
x_t = (img1_size[1]*2 - width_t) / 2
draw.text((x_t, 90), title, font=font_t, fill=(0, 0, 0))
author = '@ ' + story.author
width_a, height_a = draw.textsize(author, font_t)
x_a = (img1_size[1]*2 - width_a) / 2
draw.text((x_a, 130), author, font=font_t, fill=(0, 0, 0))
project_team = '@ Teamname'
width_team, height_team = draw.textsize(project_team, font_t)
x_team = (img1_size[1]*2 - width_team) / 2
draw.text((x_team, img1_size[0]+img2_size[0]+img3_size[0]+img4_size[1]+250), project_team, font=font_t, fill=(0, 0, 0))
text1 = story.story_in_chapters[0].split('\n')
text2 = story.story_in_chapters[1].split('\n')
text3 = story.story_in_chapters[2].split('\n')
text4 = story.story_in_chapters[3].split('\n')

width = 65
offset = 25
for i in text1:
  for line in textwrap.wrap(i, width=width):
    draw.text((img1_size[1]+25, 185+offset), line, font=font, fill=(0, 0, 0))
    offset += 25
  offset += 5

offset = 25
for i in text2:
  for line in textwrap.wrap(i, width=width):
    draw.text((img1_size[1]+25, img1_size[0]+190+offset), line, font=font, fill=(0, 0, 0))
    offset += 25
  offset += 5

offset = 25
for i in text3:
  for line in textwrap.wrap(i, width=width):
    draw.text((img1_size[1]+25,img1_size[0]+img2_size[0]+195+offset), line, font=font, fill=(0, 0, 0))
    offset += 25
  offset += 5

offset = 25
for i in text4:
  for line in textwrap.wrap(i, width=width):
    draw.text((img1_size[1]+25,img1_size[0]+img2_size[0]+img3_size[0]+200+offset), line, font=font, fill=(0, 0, 0))
    offset += 25
  offset += 5

# save the story as a image
new_img.save("My_Pokedream.jpg")